## Upload predictions from your instance segmentation model to Supervisely [tutorial]

(тут добавить картинку)

Welcome to Supervisely tutorial! 
(вкратце что тут будет происходить)
As an example we will use instance segmentation model from awesome framework Detectron2. 

(быстро пробежаться по плану)

(сделать отсылку к ноутбуку Detectron2, в духе "взяты элементы оттуда")

You can make a copy of this tutorial by "File -> Open in playground mode" and make changes there. DO NOT request access to this tutorial.

### Prepare dependencies

In [ ]:
!python -m pip install pyyaml==6.0
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!python -m pip install supervisely

In [ ]:
import torch, detectron2, supervisely as sly
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)
print("supervisely:", sly.__version__)

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
from tqdm import tqdm
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

### Prepare Supervisely

In [ ]:
# Насколько безопасно юзеру оставлять здесь свои доступы?
os.environ['SERVER_ADDRESS'] = "https://app.supervise.ly/"
os.environ['API_TOKEN'] = "" # YOUR API TOKEN


In [ ]:
api = sly.Api.from_env()
project_id = 12466 # YOUR SUPERVISELY PROJECT ID

src_project_info = api.project.get_info_by_id(project_id)
workspace_id = src_project_info.workspace_id
src_project_name = src_project_info.name
dst_project_name = f"{src_project_name}_labeled_inst_seg"

# Clone source project without existing annotations
clone_task_id = api.project.clone_advanced(
    project_id, workspace_id, dst_project_name, with_annotations=False
)
api.task.wait(clone_task_id, api.task.Status("finished"))
dst_project_info = api.project.get_info_by_name(workspace_id, dst_project_name)

# Get new project meta
project_meta_json = api.project.get_meta(dst_project_info.id)
project_meta = sly.ProjectMeta.from_json(project_meta_json)

### Prepare your model

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
    )
)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 
cfg.MODEL.DEVICE = "cpu"
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
)
predictor = DefaultPredictor(cfg)
class_names = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).get('thing_classes')

Add model classes and 'confidence' tag to project meta

In [ ]:
model_obj_classes = []
for class_name in class_names:
    model_obj_classes.append(sly.ObjClass(class_name, sly.Bitmap))
confidence_tag_meta = sly.TagMeta(
    name='confidence', 
    value_type=sly.TagValueType.ANY_NUMBER, 
    applicable_to=sly.TagApplicableTo.OBJECTS_ONLY
)
model_meta = sly.ProjectMeta(
    obj_classes=model_obj_classes, 
    tag_metas=[confidence_tag_meta]
)
merged_project_meta = project_meta.merge(model_meta)
api.project.update_meta(project_id, merged_project_meta)

### Base model usage

In [ ]:
image_id = 18274952 # YOUR IMAGE ID FROM SUPERVISELY
image = api.image.download_np(image_id)
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
cv2_imshow(image)

In [ ]:
# Get predictions from Detectron2 model
outputs = predictor(image)

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

maybe convert labels and use sly visualizer?

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

### Label your project

In [ ]:
def apply_model_to_image(image, model, class_names, project_meta):
    img_size = image.shape[:2]
    # Get predictions from Detectron2 model
    outputs = model(image)

    pred_classes = outputs["instances"].pred_classes.detach().numpy()
    pred_class_names = [class_names[pred_class] for pred_class in pred_classes]
    pred_scores = outputs["instances"].scores.detach().numpy()
    pred_masks = outputs["instances"].pred_masks.detach().numpy()

    # create Supervisely Labels from predicted objects
    labels = []
    for score, class_name, mask in zip(pred_scores, pred_class_names, pred_masks):
        if not mask.any():  # skip empty masks
            continue
        obj_class = project_meta.get_obj_class(class_name)
        conf_tag = sly.Tag(
            meta=project_meta.get_tag_meta("confidence"),
            value=round(float(score), 4),
        )
        geometry = sly.Bitmap(mask)
        mask_label = sly.Label(geometry, obj_class, sly.TagCollection([conf_tag]))
        labels.append(mask_label)

    # create Supervisely Annotation
    annotation = sly.Annotation(img_size=img_size, labels=labels)
    return annotation

In [3]:
datasets = api.dataset.get_list(dst_project_info.id)
imgs_num = sum([dataset.images_count for dataset in datasets])
with tqdm(total=imgs_num) as pbar:
    for dataset_info in datasets:
        ds_images = api.image.get_list(dataset_info.id)
        for image_info in ds_images:
            image = api.image.download_np(image_info.id)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            annotation = apply_model_to_image(
                image, predictor, class_names, merged_project_meta
            )
            api.annotation.upload_ann(image_info.id, annotation)
            pbar.update()